# Spectral filter simulation

## Instructions

To run, choose **Run->Run all cells** in the menu. Then scroll down to view the controls and the image.

## What is it?

You can use this notebook to approximate the effects of viewing scenes in the [CAVE Multispectral Image Database](https://www.cs.columbia.edu/CAVE/databases/multispectral/) in daylight through a spectral filter. Those images consist of 31 narrow band images in the 400—700 nm range.

An example of such a spectral filter could be glasses that have a notch to enhance separation between red and green colors, probably somewhere around the 550-600 nm range.

To be precise, it:

- Applies suitable lighting to the scenes (standard illuminant D65, corresponding to neutral daylight). The original scenes supposedly correspond to the reflectances of the scene, i.e. illuminated by an equal-energy radiator.
- Computes the cone cell responses (LMS space) corresponding to the illuminated scene. This can be done with modified cone cell responses, for example to simulate filters or color blindness.
- If a filter is being simulated, applies chromatic adaptation to the LMS outputs with the assumption that the eye has adapted to the filter. This means that the observer has adapted so that the color of the filter looks like D65 white.
- Computes the _standard observer_ LMS responses for monochromatic red, green and blue in order to render the scene in sRGB. This, too, could be done for any observer; however currently this functionality is not exposed in this notebook. This allows a normal-vision user to simulate color blindness, and all other conditions, with this notebook. If we computed these for a color-blind user, we could do the opposite: A color-blind observer could use it to simulate a normal-vision user, to the extent that the output colors are not out of gamut in that color blindness.

## How to use it?

The LMS response curve you see below (after running the cells) corresponds to normal vision. Notice how even in normal vision the L and M curves ("red" and "green", though these names are a bit misleading) mostly overlap each other. In the most common forms of color blindness, deuteranomalia, the M curve has shifted towards longer wavelengths. For what I believe is a reasonably bad, but not extreme, case of color blindness, try setting the M slider to 17. This shifts the M curve by 17 nanometers toward the L one. You can observe that the photo is updated to reproduce a smaller number of colors.

With the sliders below the photo, you can apply an optical filter. For example, setting the _560 nm_ slider to 0.00 would mean that all light at the 510 nm wavelength is blocked. The cone cell response curves are updated to reflect the new response. Everything is shown with normalized values, so you can apply a filter that is not practically useful because it blocks too much light, and this notebook will still happily show the result to you. I apologize for the user interface; it wasn't my top priority.

You can use the _Image_ drop down menu to choose a different scene.

The _Enable filter_ toggle enables or disables the filter settings you have set below. This is useful for seeing the effects of a filter at a glance by switching between filtered and unfiltered views.

Similarly, the _Enable shift_ toggle enables or disables the LMS shifts you have applied in the three sliders.

## Things to try out

Here are a couple of color separation optimized filter settings you might want to try out:

### Normal vision

For normal vision, ensure that the LMS shifts are set to 0. For a good color separation filter, try these settings (again, apologies for not having a button to apply them). This filter blocks about 78% of light. Normal sunglasses block about 60% to 92% of light.

- 400—420 nm: 1.00
- 430—510 nm: 0.00
- 520 nm: 1.00
- 530 nm: 0.80
- 540—610 nm: 0.00
- 620—700 nm: 1.00

Now try toggling the _Enable filter_ toggle to compare the effect with and without filter.

### Deuteranomaly

Set the _M shift_ slider to 17 nm to simulate a reasonably severe deuteranomaly. Ensure that the _Enable shift_ and _Enable filter_ toggles are on. Try the following filter settings, optimized for color separation. This filter blocks about 72% of light.

- 400—410 nm: 1.00
- 420—530 nm: 0.00
- 540—550 nm: 1.00
- 560 nm: 0.05
- 570—610 nm: 0.00
- 620—700 nm: 1.00

In [1]:
import os, numpy as np, ipywidgets as widgets

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
from ipywidgets import HBox, VBox
import ipywidgets as widgets

import render_filtered as rend

In [2]:
def plot_image(name, lms):
    plt.imshow(rend.linrgb_to_srgb(rend.load_adapted_image_linrgb(name, lms)))

In [3]:
sliders = [widgets.FloatSlider(
    min=0.0, max=1.0, value=1.0, step=0.05, continuous_update=False, description=f'{nm} nm')
    for nm in rend.IMG_WAVELENS]
enable_filter = widgets.Checkbox(value=True, description="Enable filter")
enable_shift = widgets.Checkbox(value=True, description="Enable shift")
image_chooser = widgets.Dropdown(options=sorted(os.listdir('data/')), description="Image")
lshift = widgets.IntSlider(min=-50, max=50, continuous_update=False, value=0, description="L (red) shift")
mshift = widgets.IntSlider(min=-50, max=50, continuous_update=False, value=0, description="M (green) shift")
sshift = widgets.IntSlider(min=-50, max=50, continuous_update=False, value=0, description="S (blue) shift")
lshift.layout.width = '780px'
mshift.layout.width = '780px'
sshift.layout.width = '780px'
widget_dict = {'enable_filter': enable_filter, 'enable_shift': enable_shift,
               'name': image_chooser,
               'lshift': lshift, 'mshift': mshift, 'sshift': sshift,
               **{s.description: s for s in sliders}}

In [4]:
def plot_lms(lmsp):
    fig, ax = plt.subplots(figsize=(12, 3))
    (l,) = ax.plot(rend.PREC_WAVELENS, lmsp[:, 0] / max(lmsp[:, 0]), 'r')
    (m,) = ax.plot(rend.PREC_WAVELENS, lmsp[:, 1] / max(lmsp[:, 1]), 'g')
    (s,) = ax.plot(rend.PREC_WAVELENS, lmsp[:, 2] / max(lmsp[:, 2]), 'b')
    ax.set_xlim((rend.PREC_WAVELENS.min(), 700))
    ax.xaxis.set_major_locator(MultipleLocator(50))
    ax.xaxis.set_minor_locator(MultipleLocator(10))
    ax.set_xlabel('Wavelength (nm)')
    return l, m, s

In [5]:
def calc_lms(*, lshift, mshift, sshift, enable_filter, enable_shift, **kwargs):
    lms = rend.STDLMS
    if enable_shift:
        lms = rend.shift_lms(lms, lshift, mshift, sshift)
    if enable_filter:
        lms *= rend.interp_freqs(np.asarray([s.value for s in sliders])).reshape(-1, 1)
    return lms

def update_image(*, name, **kwargs):
    plt.figure(figsize=(8, 8))
    plot_image(name, calc_lms(**kwargs))

def update_lms(**kwargs):
    plot_lms(calc_lms(**kwargs))

image = widgets.interactive_output(update_image, widget_dict)

lms_plot = widgets.interactive_output(update_lms, widget_dict)
lms_plot.layout.height = '220px'
lms_plot.layout.width = '800px'

sliders_box = HBox([VBox(sliders[:10]), VBox(sliders[10:20]), VBox(sliders[20:])])

lms_box = VBox([lms_plot, lshift, mshift, sshift])
layout = VBox([HBox([image, VBox([image_chooser, enable_filter, enable_shift])]), sliders_box])

In [6]:
disp = display(lms_box)

In [7]:
display(layout)